In [1]:
import numpy as np
from collections import defaultdict

# 주어진 그리드
grid = np.array([[1, 1], [1, 1], [1, 0]])

# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners(grid):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] == 1:
                cell_corners = [
                    (i, j),
                    (i, j+1),
                    (i+1, j+1),
                    (i+1, j)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners(grid):
    cell_corners = get_cell_corners(grid)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 폴리건의 꼭지점 좌표 계산
polygon_corners = get_polygon_corners(grid)

# 결과 출력
print("Room1 polygon corners:", polygon_corners)


Room1 polygon corners: [(0, 0), (2, 0), (2, 2), (1, 2), (1, 3), (0, 3)]


# For multi room

In [2]:
import numpy as np
from collections import defaultdict

# 주어진 그리드
grid = np.array([
    [ 4,  4,  4,  4,  1, -1, -1],
    [ 2,  2,  4,  1,  1,  5,  5],
    [ 2,  2,  3,  3,  1,  5, -1],
    [-1, -1,  3,  3, -1, -1, -1]
])
grid = np.flipud(grid)

# Add a border of -1 around the original grid
padding = 1
padded_grid = np.pad(grid, pad_width=padding, mode='constant', constant_values=-1)

# Define the scale
scale = 1000  # 1 unit = 1000mm
wall_thickness = 50  # Wall thickness in mm
# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners(grid, room_number):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] == room_number:
                cell_corners = [
                    (i, j),
                    (i, j+1),
                    (i+1, j+1),
                    (i+1, j)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners(grid, room_number):
    cell_corners = get_cell_corners(grid, room_number)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 각 방에 대한 폴리건 꼭지점 좌표 계산 및 출력
room_numbers = [1, 2, 3, 4, 5]
room_polygons = {}

for room_number in room_numbers:
    room_polygons[room_number] = get_polygon_corners(grid, room_number)

room_polygons


{1: [(4, 2), (4, 1), (5, 1), (5, 4), (4, 4), (4, 3), (3, 3), (3, 2)],
 2: [(2, 1), (2, 3), (0, 3), (0, 1)],
 3: [(4, 0), (4, 2), (2, 2), (2, 0)],
 4: [(2, 3), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (0, 4), (0, 3)],
 5: [(5, 1), (6, 1), (6, 2), (7, 2), (7, 3), (5, 3)]}

# flip y coordinate, scale, padding 적용

In [3]:
import numpy as np
from collections import defaultdict

# 주어진 그리드
grid = np.array([
    [ 4,  4,  4,  4,  1, -1, -1],
    [ 2,  2,  4,  1,  1,  5,  5],
    [ 2,  2,  3,  3,  1,  5, -1],
    [-1, -1,  3,  3, -1, -1, -1]
])

# grid를 상하 반전합니다.
grid = np.flipud(grid)

# 셀의 크기 및 패딩 설정
cell_size = 1000
padding_size = cell_size

# 패딩 적용을 위해 grid 크기를 확장합니다.
padded_grid = np.pad(grid, pad_width=1, mode='constant', constant_values=-1)

# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners(grid, room_number):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] == room_number:
                cell_corners = [
                    (i * cell_size, j * cell_size),
                    (i * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, j * cell_size)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners(grid, room_number):
    cell_corners = get_cell_corners(grid, room_number)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 각 방에 대한 폴리건 꼭지점 좌표 계산 및 출력
room_numbers = [1, 2, 3, 4, 5]
room_polygons = {}

for room_number in room_numbers:
    room_polygons[room_number] = get_polygon_corners(padded_grid, room_number)

room_polygons


{1: [(5000, 3000),
  (5000, 2000),
  (6000, 2000),
  (6000, 5000),
  (5000, 5000),
  (5000, 4000),
  (4000, 4000),
  (4000, 3000)],
 2: [(3000, 2000), (3000, 4000), (1000, 4000), (1000, 2000)],
 3: [(5000, 1000), (5000, 3000), (3000, 3000), (3000, 1000)],
 4: [(3000, 4000),
  (3000, 3000),
  (4000, 3000),
  (4000, 4000),
  (5000, 4000),
  (5000, 5000),
  (1000, 5000),
  (1000, 4000)],
 5: [(6000, 2000),
  (7000, 2000),
  (7000, 3000),
  (8000, 3000),
  (8000, 4000),
  (6000, 4000)]}

In [4]:
import numpy as np
from collections import defaultdict
import ezdxf

# 주어진 그리드
grid = np.array([
    [ 4,  4,  4,  4,  1, -1, -1],
    [ 2,  2,  4,  1,  1,  5,  5],
    [ 2,  2,  3,  3,  1,  5, -1],
    [-1, -1,  3,  3, -1, -1, -1]
])

# grid를 상하 반전합니다.
grid = np.flipud(grid)

# 셀의 크기 및 패딩 설정
cell_size = 1000
padding_size = cell_size

# 패딩 적용을 위해 grid 크기를 확장합니다.
padded_grid = np.pad(grid, pad_width=1, mode='constant', constant_values=-1)

# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners(grid, room_number):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] == room_number:
                cell_corners = [
                    (i * cell_size, j * cell_size),
                    (i * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, j * cell_size)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners(grid, room_number):
    cell_corners = get_cell_corners(grid, room_number)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 각 방에 대한 폴리건 꼭지점 좌표 계산 및 출력
room_numbers = [1, 2, 3, 4, 5]
room_polygons = {}

for room_number in room_numbers:
    room_polygons[room_number] = get_polygon_corners(padded_grid, room_number)

# DXF 파일 생성 함수
def save_polygons_to_dxf(room_polygons, filename):
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()

    for room_number, corners in room_polygons.items():
        polyline = msp.add_lwpolyline(corners, close=True)
        polyline.dxf.layer = f"Room_{room_number}"

    doc.saveas(filename)

# DXF 파일 저장
output_file = "room_polygons.dxf"
save_polygons_to_dxf(room_polygons, output_file)
print(f"DXF file saved as {output_file}")


DXF file saved as room_polygons.dxf


# 각 방의 폴리건을 lwpolyline dxf 파일에 저장

In [5]:
import numpy as np
from collections import defaultdict
import ezdxf

# 주어진 그리드
grid = np.array([
    [ 4,  4,  4,  4,  1, -1, -1],
    [ 2,  2,  4,  1,  1,  5,  5],
    [ 2,  2,  3,  3,  1,  5, -1],
    [-1, -1,  3,  3, -1, -1, -1]
])

# grid를 상하 반전합니다.
grid = np.flipud(grid)

# 셀의 크기 및 패딩 설정
cell_size = 1000
padding_size = cell_size

# 패딩 적용을 위해 grid 크기를 확장합니다.
padded_grid = np.pad(grid, pad_width=1, mode='constant', constant_values=-1)

# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners(grid, room_number):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] == room_number:
                cell_corners = [
                    (i * cell_size, j * cell_size),
                    (i * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, j * cell_size)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners(grid, room_number):
    cell_corners = get_cell_corners(grid, room_number)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 각 방에 대한 폴리건 꼭지점 좌표 계산 및 출력
room_numbers = [1, 2, 3, 4, 5]
room_polygons = {}

for room_number in room_numbers:
    room_polygons[room_number] = get_polygon_corners(padded_grid, room_number)

# DXF 파일 생성 함수
def save_polygons_to_dxf(room_polygons, filename):
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()

    for room_number, corners in room_polygons.items():
        polyline = msp.add_lwpolyline(corners, close=True)
        polyline.dxf.layer = f"Room_{room_number}"

    doc.saveas(filename)

# DXF 파일 저장
output_file = "room_polygons.dxf"
save_polygons_to_dxf(room_polygons, output_file)
print(f"DXF file saved as {output_file}")


DXF file saved as room_polygons.dxf


In [8]:
# 주어진 그리드
grid = np.array([
    [ 4,  4,  4,  4,  1, -1, -1],
    [ 2,  2,  4,  1,  1,  5,  5],
    [ 2,  2,  3,  3,  1,  5, -1],
    [-1, -1,  3,  3, -1, -1, -1]
])

# grid를 상하 반전합니다.
grid = np.flipud(grid)

# 셀의 크기 및 패딩 설정
cell_size = 1000
padding_size = cell_size

# 패딩 적용을 위해 grid 크기를 확장합니다.
padded_grid = np.pad(grid, pad_width=1, mode='constant', constant_values=-1)

# 셀의 꼭지점 좌표를 구하는 함수
def get_cell_corners_all(grid):
    rows, cols = grid.shape
    corners = []
    for i in range(cols):
        for j in range(rows):
            if grid[j, i] != -1:
                cell_corners = [
                    (i * cell_size, j * cell_size),
                    (i * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, (j + 1) * cell_size),
                    ((i + 1) * cell_size, j * cell_size)
                ]
                corners.append(cell_corners)
    return corners

# 외곽선 좌표를 구하는 함수
def get_polygon_corners_all(grid):
    cell_corners = get_cell_corners_all(grid)
    corner_count = defaultdict(int)
    
    # 각 꼭지점의 등장 횟수를 셉니다.
    for cell in cell_corners:
        for corner in cell:
            corner_count[corner] += 1
            
    # 외곽선에 위치한 꼭지점만 선택합니다.
    outer_corners = [corner for corner, count in corner_count.items() if count < 4]

    # 시계 방향으로 정렬합니다.
    def sort_corners(corners):
        center = np.mean(corners, axis=0)
        corners.sort(key=lambda c: np.arctan2(c[1] - center[1], c[0] - center[0]))
        return corners

    sorted_corners = sort_corners(outer_corners)

    # 일직선 상에 있는 점들을 제거합니다.
    def remove_collinear_points(corners):
        def is_collinear(p1, p2, p3):
            return (p2[1] - p1[1]) * (p3[0] - p2[0]) == (p3[1] - p2[1]) * (p2[0] - p1[0])
        
        filtered_corners = []
        n = len(corners)
        for i in range(n):
            p1 = corners[i]
            p2 = corners[(i + 1) % n]
            p3 = corners[(i + 2) % n]
            if not is_collinear(p1, p2, p3):
                filtered_corners.append(p2)
        return filtered_corners

    final_corners = remove_collinear_points(sorted_corners)
    return final_corners

# 전체 그리드에서 -1을 제외한 모든 셀로 구성된 폴리건의 좌표 계산
all_room_polygon = get_polygon_corners_all(padded_grid)



# DXF 파일 생성 함수
def save_polygon_to_dxf(polygon, filename):
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()

    polyline = msp.add_lwpolyline(polygon, close=True)
    polyline.dxf.layer = "All_Rooms"

    doc.saveas(filename)

# DXF 파일 저장
output_file = "all_room_polygon.dxf"
save_polygon_to_dxf(all_room_polygon, output_file)
print(f"DXF file saved as {output_file}")


DXF file saved as all_room_polygon.dxf


# 수직인지 결정하는 함수
1. Shape 클래스: 코너 좌표를 저장하는 클래스입니다.
2. is_perpendicular 함수: 주어진 corner_index에 대해 해당 코너가 앞뒤 코너와 수직인지를 결정합니다.
    - prev_corner: 이전 코너의 좌표.
    - current_corner: 현재 코너의 좌표.
    - next_corner: 다음 코너의 좌표.
    - 벡터 계산: 이전 코너와 현재 코너 간의 벡터(prev_vector), 현재 코너와 다음 코너 간의 벡터(next_vector)를 계산합니다.
    - 점곱 계산: 두 벡터의 점곱을 계산합니다. 점곱이 0이면 두 벡터는 수직입니다.
### 수직성 판별:
- 이 함수는 주어진 코너가 이전 코너와 다음 코너와 각각 수직인지를 Boolean 값으로 반환합니다.

In [12]:
class Shape:
    def __init__(self, corners):
        self.corners = corners

    def is_corner_perpendicular(self, corner_index):
        num_corners = len(self.corners)
        
        # Get the previous, current, and next corners
        prev_corner = self.corners[(corner_index - 1) % num_corners]
        current_corner = self.corners[corner_index]
        next_corner = self.corners[(corner_index + 1) % num_corners]

        # Calculate vectors
        prev_vector = (current_corner[0] - prev_corner[0], current_corner[1] - prev_corner[1])
        next_vector = (next_corner[0] - current_corner[0], next_corner[1] - current_corner[1])

        # Check for perpendicularity by calculating the dot product
        dot_product = prev_vector[0] * next_vector[0] + prev_vector[1] * next_vector[1]
        
        # If the dot product is zero, the vectors are perpendicular
        return dot_product == 0

# Example usage
shape = Shape([(5000, 1000),
 (5000, 2000),
 (4000, 2000),
 (4000, 3500),
 (3000, 3500),
 (3000, 2000),
 (1000, 2000),
 (1000, 1000)])
print(shape.is_corner_perpendicular(1))  # Example output


True


In [28]:
class Shape:
    def __init__(self, corners):
        # Convert tuples to lists to allow modification
        self.corners = [list(corner) for corner in corners]

    def sync_adjacent_corner(self, corner_index, updated_axis):
        num_corners = len(self.corners)
        
        # Get the previous, current, and next corners
        prev_corner = self.corners[(corner_index - 1) % num_corners]
        current_corner = self.corners[corner_index]
        next_corner = self.corners[(corner_index + 1) % num_corners]

        # Check alignment with previous and next corners
        prev_aligned = (prev_corner[0] == current_corner[0] or prev_corner[1] == current_corner[1])
        next_aligned = (next_corner[0] == current_corner[0] or next_corner[1] == current_corner[1])

        if prev_aligned and next_aligned:
            return True  # Both previous and next corners are aligned

        # If not aligned, update the specified axis to match the current corner
        if not prev_aligned:
            if updated_axis == 'x':
                self.corners[(corner_index - 1) % num_corners][0] = current_corner[0]
            else:
                self.corners[(corner_index - 1) % num_corners][1] = current_corner[1]

        if not next_aligned:
            if updated_axis == 'x':
                self.corners[(corner_index + 1) % num_corners][0] = current_corner[0]
            else:
                self.corners[(corner_index + 1) % num_corners][1] = current_corner[1]

        return False  # At least one corner was not aligned

# Example usage
shape = Shape([(5000, 2000),
 (5000, 1000),
 (6000, 1000),
 (6000, 4000),
 (5000, 4000),
 (5000, 3000),
 (4000, 3500),
 (4000, 2000)])
print(shape.sync_adjacent_corner(6, 'y'))  # Example output
print(shape.corners)  # Updated corners


False
[[5000, 2000], [5000, 1000], [6000, 1000], [6000, 4000], [5000, 4000], [5000, 3500], [4000, 3500], [4000, 2000]]


In [31]:
from shapely.geometry import LineString, Point, Polygon, MultiPoint, MultiLineString

class RoomPolygon:
    def __init__(self, corners):
        self.corners = corners
        self.polygon = Polygon(corners)
        self.area = self.calculate_area()
        self.perimeter = self.calculate_perimeter()
        self.min_length = self.calculate_min_length()
        self.simplicity = self.calculate_simplicity()
        self.moved_corners = []  # 이동된 코너들을 추적합니다.

    def calculate_area(self):
        return self.polygon.area

    def calculate_perimeter(self):
        return self.polygon.length

    def calculate_min_length(self):
        return min([LineString([self.corners[i], self.corners[(i + 1) % len(self.corners)]]).length for i in range(len(self.corners))])

    def calculate_simplicity(self):
        return self.area / self.perimeter

    def get_intersections(self, moved_edge, other_polygon):
        intersections = []
        moved_line = LineString(moved_edge)
        other_corners = other_polygon.corners  # Accessing corners from the other polygon

        for i in range(len(other_corners)):
            edge = LineString([other_corners[i], other_corners[(i + 1) % len(other_corners)]])
            if moved_line.intersects(edge):
                intersection = moved_line.intersection(edge)
                if isinstance(intersection, Point):
                    intersections.append((intersection.x, intersection.y))
                elif isinstance(intersection, LineString):
                    intersections.extend([(point[0], point[1]) for point in intersection.coords])
                elif isinstance(intersection, MultiPoint):
                    intersections.extend([(point.x, point.y) for point in intersection])
                elif isinstance(intersection, MultiLineString):
                    for geom in intersection:
                        intersections.extend([(point[0], point[1]) for point in geom.coords])
                elif intersection.geom_type == 'GeometryCollection':
                    for geom in intersection:
                        if isinstance(geom, (Point, LineString)):
                            intersections.extend([(point[0], point[1]) for point in geom.coords])
        return intersections

# Example usage
moved_edge = [(4000, 3500), (3000, 3500)]
other_polygon = RoomPolygon([(5000, 3500), (5000, 5000), (3000, 5000), (3000, 3500)])

room_polygon = RoomPolygon([(4000, 3000), (4000, 3500), (3000, 3500), (3000, 3000)])
intersections = room_polygon.get_intersections(moved_edge, other_polygon)
print(intersections)


[(3000.0, 3500.0), (4000.0, 3500.0), (3000.0, 3500.0)]


corners 가 바뀔때마다 자동으로 재계산

In [34]:
from shapely.geometry import Polygon
from shapely.geometry.polygon import LinearRing

class RoomPolygon:
    def __init__(self, corners):
        self._corners = None
        self.corners = corners

    @property
    def corners(self):
        return self._corners

    @corners.setter
    def corners(self, new_corners):
        self._corners = new_corners
        self.polygon = Polygon(new_corners)
        self.area = self.calculate_area()
        self.perimeter = self.calculate_perimeter()
        self.min_length = self.calculate_min_length()
        self.simplicity = self.calculate_simplicity()

    def calculate_area(self):
        return self.polygon.area

    def calculate_perimeter(self):
        return self.polygon.length

    def calculate_min_length(self):
        # Calculate the shortest side length
        ring = LinearRing(self.polygon.exterior.coords)
        min_length = min(ring.project(ring.interpolate(distance))
                         for distance in range(int(ring.length)))
        return min_length

    def calculate_simplicity(self):
        # Simplicity calculation logic (example: simple if no self-intersections)
        return self.polygon.is_valid and self.polygon.is_simple

# 사용 예제
corners = [(0, 0), (4, 0), (4, 3), (0, 3)]
room = RoomPolygon(corners)

print(f"Area: {room.area}")
print(f"Perimeter: {room.perimeter}")
print(f"Min Length: {room.min_length}")
print(f"Simplicity: {room.simplicity}")

# corners 변경
new_corners = [(0, 0), (5, 0), (5, 4), (0, 4)]
room.corners = new_corners

print("Corners updated.")
print(f"New Area: {room.area}")
print(f"New Perimeter: {room.perimeter}")
print(f"New Min Length: {room.min_length}")
print(f"New Simplicity: {room.simplicity}")


Area: 12.0
Perimeter: 14.0
Min Length: 0.0
Simplicity: True
Corners updated.
New Area: 20.0
New Perimeter: 18.0
New Min Length: 0.0
New Simplicity: True


In [33]:
from shapely.geometry import Polygon
from shapely.geometry.polygon import LinearRing

class RoomPolygon:
    def __init__(self, corners):
        self._corners = None
        self.corners = corners

    @property
    def corners(self):
        return self._corners

    @corners.setter
    def corners(self, new_corners):
        self._corners = new_corners
        self.polygon = Polygon(new_corners)
        self.area = self.calculate_area()
        self.perimeter = self.calculate_perimeter()
        self.min_length = self.calculate_min_length()
        self.simplicity = self.calculate_simplicity()

    def calculate_area(self):
        return self.polygon.area

    def calculate_perimeter(self):
        return self.polygon.length

    def calculate_min_length(self):
        # Calculate the shortest side length
        ring = LinearRing(self.polygon.exterior.coords)
        min_length = min(ring.project(ring.interpolate(distance))
                         for distance in range(int(ring.length)))
        return min_length

    def calculate_simplicity(self):
        # Simplicity calculation logic (example: simple if no self-intersections)
        return self.polygon.is_valid and self.polygon.is_simple

# 사용 예제
corners = [(0, 0), (4, 0), (4, 3), (0, 3)]
room = RoomPolygon(corners)

print(f"Area: {room.area}")
print(f"Perimeter: {room.perimeter}")
print(f"Min Length: {room.min_length}")
print(f"Simplicity: {room.simplicity}")

# corners 변경
new_corners = [(0, 0), (5, 0), (5, 4), (0, 4)]
room.corners = new_corners

print("Corners updated.")
print(f"New Area: {room.area}")
print(f"New Perimeter: {room.perimeter}")
print(f"New Min Length: {room.min_length}")
print(f"New Simplicity: {room.simplicity}")


Area: 12.0
Perimeter: 14.0
Min Length: 0.0
Simplicity: True
Corners updated.
New Area: 20.0
New Perimeter: 18.0
New Min Length: 0.0
New Simplicity: True


In [35]:
from shapely.geometry import Polygon
from shapely.geometry.polygon import LinearRing

class RoomPolygon:
    def __init__(self, corners):
        self._corners = None
        self.corners = corners  # setter가 호출됨

    @property
    def corners(self):
        return self._corners

    @corners.setter
    def corners(self, new_corners):
        self._corners = new_corners
        self.polygon = Polygon(new_corners)
        self.area = self.calculate_area()
        self.perimeter = self.calculate_perimeter()
        self.min_length = self.calculate_min_length()
        self.simplicity = self.calculate_simplicity()

    def calculate_area(self):
        return self.polygon.area

    def calculate_perimeter(self):
        return self.polygon.length

    def calculate_min_length(self):
        # Calculate the shortest side length
        ring = LinearRing(self.polygon.exterior.coords)
        coords = list(ring.coords)
        min_length = float('inf')
        for i in range(len(coords) - 1):
            length = ((coords[i][0] - coords[i + 1][0]) ** 2 + (coords[i][1] - coords[i + 1][1]) ** 2) ** 0.5
            if length < min_length:
                min_length = length
        return min_length

    def calculate_simplicity(self):
        # Simplicity calculation logic (example: simple if no self-intersections)
        return self.polygon.is_valid and self.polygon.is_simple

# 예제 함수: 공통 변 업데이트
def update_shared_edge(room1, room2, new_shared_edge):
    # 새로운 코너 좌표를 반영하도록 업데이트
    room1.corners = [(0, 0), new_shared_edge[0], new_shared_edge[1], (0, 4)]
    room2.corners = [new_shared_edge[0], (5, 0), (5, 4), new_shared_edge[1]]

# RoomPolygon 인스턴스 생성
room1 = RoomPolygon([(0, 0), (4, 0), (4, 4), (0, 4)])
room2 = RoomPolygon([(4, 0), (8, 0), (8, 4), (4, 4)])

print(f"Room1 Area: {room1.area}, Perimeter: {room1.perimeter}, Min Length: {room1.min_length}, Simplicity: {room1.simplicity}")
print(f"Room2 Area: {room2.area}, Perimeter: {room2.perimeter}, Min Length: {room2.min_length}, Simplicity: {room2.simplicity}")

# 공통 변 업데이트
update_shared_edge(room1, room2, [(4, 0), (4, 4)])

print("Updated Room1 and Room2")

print(f"Room1 Area: {room1.area}, Perimeter: {room1.perimeter}, Min Length: {room1.min_length}, Simplicity: {room1.simplicity}")
print(f"Room2 Area: {room2.area}, Perimeter: {room2.perimeter}, Min Length: {room2.min_length}, Simplicity: {room2.simplicity}")


Room1 Area: 16.0, Perimeter: 16.0, Min Length: 4.0, Simplicity: True
Room2 Area: 16.0, Perimeter: 16.0, Min Length: 4.0, Simplicity: True
Updated Room1 and Room2
Room1 Area: 16.0, Perimeter: 16.0, Min Length: 4.0, Simplicity: True
Room2 Area: 4.0, Perimeter: 10.0, Min Length: 1.0, Simplicity: True


# outer_corner를 구하는 방법



In [82]:
corners = [(5000, 1000),
 (6000, 2000),
 (6000, 1000),
 (5000, 3000),
 (6000, 3000),
 (7000, 3000),
 (6000, 4000),
 (7000, 4000),
 (8000, 3000),
 (8000, 2000)]

In [100]:
outers=[]
next_rotation = 'horizontal'
min_y = min(corners, key=lambda c:c[1])[1]
minx_miny = min((c for c in corners if c[1] == min_y ), key=lambda c: c[0])
maxx_miny = max((c for c in corners if c[1]==min_y), key = lambda c: c[0])

outers.append(minx_miny)
outers.append(maxx_miny)

next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
y_points = [c for c in corners if c[0]==outers[len(outers)-1][0]]
y_points.sort(key=lambda c: c[1])
y_points.reverse()


start = y_points.pop()
end = y_points.pop()
print(f'horizontal_line = {start},{end}')
x_start = end[0]
y_start = end[1]
print(f'start = {start}, end={end}')
x_points = [c for c in corners if c[1] == end[1]][0]# end의 y점과 같은 x 좌표
vertical_start  = end
(start, end) , x_points 
x_parallels = [c for c in corners if c[1] == 2000 and c[0] > end[0]] # x 좌표가 현재 x 좌표보다 큰 corner들 
y_end = y_start
x_end = min(x_parallels)[0]
vertical_end = (x_end, y_end)
vertical_start, vertical_end

horizontal_line = (6000, 1000),(6000, 2000)
start = (6000, 1000), end=(6000, 2000)


((6000, 2000), (8000, 2000))

In [121]:
import numpy as np
from collections import deque
outers=[]
next_rotation = 'horizontal'
# miny의 x 값 중 최소
miny = np.min(corners, axis=0)[1] #1000
miny_parallels = [c for c in corners if c[1] == miny]
miny_parallels.sort(key=lambda c:c[0], reverse=True) #descending order

horiz_start = miny_parallels.pop()
miny_parallels.append((7000,1000))#for test
horiz_end = min(miny_parallels, key=lambda c: c[0])
outers.append((horiz_start, horiz_end))
outers



[((5000, 1000), (6000, 1000))]

In [122]:
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'


In [112]:

horiz_parallels = [c for c in corners if c[1] == horiz_start[1] and c[0] > horiz_start[0]]
horiz_parallels.append((7000,1000) )
horiz_parallels.append((8000, 500))
np.min(horiz_parallels, axis=0)

array([6000,  500])

In [102]:

maxx_miny = max((c for c in corners if c[1]== horiz_start[0]), key = lambda c: c[0])
horiz_end =
min_y = min(corners, key=lambda c:c[1])[1]
minx_miny = min((c for c in corners if c[1] == min_y ), key=lambda c: c[0])
print(top_left, minx_miny, maxx_miny)

outers.append(minx_miny)
outers.append(maxx_miny)

next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
y_points = [c for c in corners if c[0]==outers[len(outers)-1][0]]
y_points.sort(key=lambda c: c[1])
y_points.reverse()


start = y_points.pop()
end = y_points.pop()
print(f'horizontal_line = {start},{end}')
x_start = end[0]
y_start = end[1]
print(f'start = {start}, end={end}')
x_points = [c for c in corners if c[1] == end[1]][0]# end의 y점과 같은 x 좌표
vertical_start  = end
(start, end) , x_points 
x_parallels = [c for c in corners if c[1] == 2000 and c[0] > end[0]] # x 좌표가 현재 x 좌표보다 큰 corner들 
y_end = y_start
x_end = min(x_parallels)[0]
vertical_end = (x_end, y_end)
vertical_start, vertical_end

[5000 1000] (5000, 1000) (6000, 1000)
horizontal_line = (6000, 1000),(6000, 2000)
start = (6000, 1000), end=(6000, 2000)


((6000, 2000), (8000, 2000))

In [195]:
corners = [(5000, 1000),
 (6000, 2000),
 (6000, 1000),
 (5000, 3000),
 (6000, 3000),
 (7000, 3000),
 (6000, 4000),
 (7000, 4000),
 (8000, 3000),
 (8000, 2000)]

In [210]:
corners=[(1000,2000),(3000,2000),(3000,3000), (2000,3000), (2000,4000), (4000, 4000), (4000, 3000), (5000,3000), (5000,5000),(1000,5000)]

In [211]:
# first horizontal
import numpy as np
from collections import deque
outers=[]
next_rotation = 'horizontal'
# miny의 x 값 중 최소
miny = np.min(corners, axis=0)[1] #1000
miny_parallels = [c for c in corners if c[1] == miny]
miny_parallels.sort(key=lambda c:c[0], reverse=True) #descending order

horiz_start = miny_parallels.pop()
horiz_end = min(miny_parallels, key=lambda c: c[0])
last = horiz_end
outers.append((horiz_start, horiz_end))
outers



[((1000, 2000), (3000, 2000))]

In [212]:
# first vertical
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
vert_start = last
print(f'vert_start={vert_start}')
lastx_parallels = [c for c in corners if c[0] == vert_start[0] and c[1] > vert_start[1]] #x 축이 같은  y,  세로로 직선을 그어 만나면 거기서 end
lastx_parallels.sort(key = lambda c:c[1], reverse=True)
print(f'lastx_parallels = {lastx_parallels}')
vert_end = lastx_parallels.pop()
last = vert_end
outers.append((vert_start,vert_end))
print(f'outers = {outers}')


vert_start=(3000, 2000)
lastx_parallels = [(3000, 3000)]
outers = [((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000))]


In [213]:
# next_ horizontal
# 왼쪽으로 갈 수도 있고 오른쪽으로 갈 수도 있다. 어떻게 하나 홀수개가 있는 쪽으로 이동한다.
import math
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
horiz_start = last
print(f'horiz_start = {horiz_start}')
horiz_points = [c for c in corners if c[1] == horiz_start[1] ] 
lefts = [c for c in horiz_points if c[0] < horiz_start[0]]
rights = [c for c in horiz_points if c[0] > horiz_start[0]]

# 홀수 개수인 집합 고르기
if len(lefts) % 2 != 0:
    odd_set = lefts
else:
    odd_set = rights
    
if len(odd_set) ==  1: #  하나이면 그냥 그거
    horiz_end = odd_set[0]
else:
    horiz_end = min(odd_set, key=lambda point: math.sqrt((point[0] - horiz_start[0])**2 + (point[1] - horiz_start[1])**2))


outers.append((horiz_start, horiz_end))
outers

horiz_start = (3000, 3000)


[((1000, 2000), (3000, 2000)),
 ((3000, 2000), (3000, 3000)),
 ((3000, 3000), (2000, 3000))]

In [214]:
# next_vertical
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
vert_start = horiz_end

vert_points = [c for c in corners if c[0] == vert_start[0]] #x 축이 같은  y,  세로로 직선을 그어 만나면 거기서 end
vert_points.sort(key = lambda c:c[1], reverse=True)
print(f'vert_points = {vert_points}')
uppers = [c for c in vert_points if c[1] < vert_start[1]]
unders = [c for c in vert_points if c[1] > vert_start[1]]
print(f'uppers={uppers}')
print(f'unders={unders}')

# 홀수 개수인 집합 고르기
if len(uppers) % 2 != 0:
    odd_set = uppers
else:
    odd_set = unders

print(f'odd_set={odd_set}')
if len(odd_set) ==  1: #  하나이면 그냥 그거
    vert_end = odd_set[0]
    
else:
    vert_end = min(odd_set, key=lambda point: math.sqrt((point[0] - vert_start[0])**2 + (point[1] - vert_start[1])**2))

print(f'vert_end={vert_end}')
last = vert_end
outers.append((vert_start,vert_end))
print(f'outers = {outers}')

vert_points = [(2000, 4000), (2000, 3000)]
uppers=[]
unders=[(2000, 4000)]
odd_set=[(2000, 4000)]
vert_end=(2000, 4000)
outers = [((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000)), ((3000, 3000), (2000, 3000)), ((2000, 3000), (2000, 4000))]


In [215]:
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
horiz_start = last
print(f'horiz_start = {horiz_start}')
horiz_points = [c for c in corners if c[1] == horiz_start[1] ] 
lefts = [c for c in horiz_points if c[0] < horiz_start[0]]
rights = [c for c in horiz_points if c[0] > horiz_start[0]]

# 홀수 개수인 집합 고르기
if len(lefts) % 2 != 0:
    odd_set = lefts
else:
    odd_set = rights
    
if len(odd_set) ==  1: #  하나이면 그냥 그거
    horiz_end = odd_set[0]
else:
    horiz_end = min(odd_set, key=lambda point: math.sqrt((point[0] - horiz_start[0])**2 + (point[1] - horiz_start[1])**2))


outers.append((horiz_start, horiz_end))
outers

horiz_start = (2000, 4000)


[((1000, 2000), (3000, 2000)),
 ((3000, 2000), (3000, 3000)),
 ((3000, 3000), (2000, 3000)),
 ((2000, 3000), (2000, 4000)),
 ((2000, 4000), (4000, 4000))]

In [193]:
vert_start[0], [c for c in corners if c[0] == vert_start[0]]

(4000, [(4000, 4000), (4000, 3000)])

In [216]:
# next_vertical
next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
vert_start = horiz_end

vert_points = [c for c in corners if c[0] == vert_start[0]] #x 축이 같은  y,  세로로 직선을 그어 만나면 거기서 end
vert_points.sort(key = lambda c:c[1], reverse=True)
print(f'vert_points = {vert_points}')
uppers = [c for c in vert_points if c[1] < vert_start[1]]
unders = [c for c in vert_points if c[1] > vert_start[1]]


# 홀수 개수인 집합 고르기
if len(uppers) % 2 != 0:
    odd_set = uppers
else:
    odd_set = unders
    
if len(odd_set) ==  1: #  하나이면 그냥 그거
    vert_end = odd_set[0]
else:
    vert_end = min(odd_set, key=lambda point: math.sqrt((point[0] - vert_start[0])**2 + (point[1] - vert_start[1])**2))

print(f'vert_end={vert_end}')
last = vert_end
outers.append((vert_start,vert_end))
print(f'outers = {outers}')

vert_points = [(4000, 4000), (4000, 3000)]
vert_end=(4000, 3000)
outers = [((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000)), ((3000, 3000), (2000, 3000)), ((2000, 3000), (2000, 4000)), ((2000, 4000), (4000, 4000)), ((4000, 4000), (4000, 3000))]


In [235]:
def add_horizontal_edge(corners, next_rotation, last):
    next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
    horiz_start = last
    # print(f'horiz_start = {horiz_start}')
    horiz_points = [c for c in corners if c[1] == horiz_start[1] ] 
    lefts = [c for c in horiz_points if c[0] < horiz_start[0]]
    rights = [c for c in horiz_points if c[0] > horiz_start[0]]
    
    # 홀수 개수인 집합 고르기
    if len(lefts) % 2 != 0:
        odd_set = lefts
    else:
        odd_set = rights
        
    if len(odd_set) ==  1: #  하나이면 그냥 그거
        horiz_end = odd_set[0]
    else:
        horiz_end = min(odd_set, key=lambda point: math.sqrt((point[0] - horiz_start[0])**2 + (point[1] - horiz_start[1])**2))
    
    print(f'horiz_start, horiz_end = ({horiz_start}, {horiz_end})')
    # outers.append((horiz_start, horiz_end))
    # print(f'outers={outers}')
    return (horiz_start, horiz_end)
    
    
def add_vertical_edge(corners, next_rotation, last):
        # next_vertical
    next_rotation = 'vertical' if next_rotation == 'horizontal' else 'horizontal'
    vert_start = last
    
    vert_points = [c for c in corners if c[0] == vert_start[0]] #x 축이 같은  y,  세로로 직선을 그어 만나면 거기서 end
    vert_points.sort(key = lambda c:c[1], reverse=True)
    # print(f'vert_points = {vert_points}')
    uppers = [c for c in vert_points if c[1] < vert_start[1]]
    unders = [c for c in vert_points if c[1] > vert_start[1]]
    
    
    # 홀수 개수인 집합 고르기
    if len(uppers) % 2 != 0:
        odd_set = uppers
    else:
        odd_set = unders
        
    if len(odd_set) ==  1: #  하나이면 그냥 그거
        vert_end = odd_set[0]
    else:
        vert_end = min(odd_set, key=lambda point: math.sqrt((point[0] - vert_start[0])**2 + (point[1] - vert_start[1])**2))
    
    print(f'vert_end={vert_end}')
    # last = vert_end
    # outers.append((vert_start,vert_end))
    # print(f'outers = {outers}')
    return (vert_start, vert_end)

In [240]:
corners=[(1000,2000),(3000,2000),(3000,3000), (2000,3000), (2000,4000), (4000, 4000), (4000, 3000), (5000,3000), (5000,5000),(1000,5000)]
next_rotation = 'vertical'

topy = min(corners,key=lambda coord: coord[1])[1] #top
print(topy)
top_left = min([c for c in corners if c[1] == miny])
last = top_left

outers = []
for i in range(int(len(corners)/2)):
    points = add_horizontal_edge(corners, next_rotation, last)
    last = points[1]
    outers.append(points)
    print(f'points={points},\tlast={last}')
    print(f'outers={outers}')
    points = add_vertical_edge(corners, next_rotation, last)
    last = points[1]
    outers.append(points)
    print(f'points={points},\tlast={last}')
    print(f'outers={outers}')




# horiz_end = min(miny_parallels, key=lambda c: c[0])
# last = horiz_end

2000
horiz_start, horiz_end = ((1000, 2000), (3000, 2000))
points=((1000, 2000), (3000, 2000)),	last=(3000, 2000)
outers=[((1000, 2000), (3000, 2000))]
vert_end=(3000, 3000)
points=((3000, 2000), (3000, 3000)),	last=(3000, 3000)
outers=[((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000))]
horiz_start, horiz_end = ((3000, 3000), (2000, 3000))
points=((3000, 3000), (2000, 3000)),	last=(2000, 3000)
outers=[((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000)), ((3000, 3000), (2000, 3000))]
vert_end=(2000, 4000)
points=((2000, 3000), (2000, 4000)),	last=(2000, 4000)
outers=[((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000)), ((3000, 3000), (2000, 3000)), ((2000, 3000), (2000, 4000))]
horiz_start, horiz_end = ((2000, 4000), (4000, 4000))
points=((2000, 4000), (4000, 4000)),	last=(4000, 4000)
outers=[((1000, 2000), (3000, 2000)), ((3000, 2000), (3000, 3000)), ((3000, 3000), (2000, 3000)), ((2000, 3000), (2000, 4000)), ((2000, 4000), (4000, 4000))]
vert_end=(4000, 3000)
po

In [238]:
outers

[((1000, 2000), (3000, 2000)),
 ((3000, 2000), (3000, 3000)),
 ((3000, 3000), (2000, 3000)),
 ((2000, 3000), (2000, 4000)),
 ((2000, 4000), (4000, 4000)),
 ((4000, 4000), (4000, 3000)),
 ((4000, 3000), (5000, 3000)),
 ((5000, 3000), (5000, 5000)),
 ((5000, 5000), (1000, 5000)),
 ((1000, 5000), (1000, 2000)),
 ((1000, 2000), (3000, 2000)),
 ((3000, 2000), (3000, 3000)),
 ((3000, 3000), (2000, 3000)),
 ((2000, 3000), (2000, 4000)),
 ((2000, 4000), (4000, 4000)),
 ((4000, 4000), (4000, 3000)),
 ((4000, 3000), (5000, 3000)),
 ((5000, 3000), (5000, 5000)),
 ((5000, 5000), (1000, 5000)),
 ((1000, 5000), (1000, 2000))]

In [246]:
import numpy as np
import random

# Function to randomly place k colors within the floorplan
def place_k_colors_on_grid(grid_arr, k):
    colors_placed = 0
    cells_coords = set()
    coloring_grid = grid_arr.copy()
    m, n = grid_arr.shape
    while colors_placed < k:
        row, col = random.randint(0, m - 1), random.randint(0, n - 1)
        if coloring_grid[row, col] == 0:  # Ensure the cell is within the floorplan and uncolored
            coloring_grid[row, col] = colors_placed + 1
            cells_coords.add((row, col))
            colors_placed += 1
    coloring_grid, cells_coords = relocate_by_orientation(coloring_grid, cells_coords)
    return coloring_grid, cells_coords

# Function to relocate cells based on orientation requirements
def relocate_by_orientation(grid, cells_coords):
    orientation_requirements = {
        1: 'north',
        2: 'south',
        3: 'east',
        4: 'west',
        5: 'north'
    }

    def get_new_position(m, n, orientation, occupied):
        attempts = 0
        max_attempts = m * n  # Limit to avoid infinite loop
        while attempts < max_attempts:
            if orientation == 'north':
                new_pos = (0, random.randint(0, n - 1))
            elif orientation == 'south':
                new_pos = (m - 1, random.randint(0, n - 1))
            elif orientation == 'east':
                new_pos = (random.randint(0, m - 1), n - 1)
            elif orientation == 'west':
                new_pos = (random.randint(0, m - 1), 0)
            if new_pos not in occupied and grid[new_pos[0], new_pos[1]] == 0:
                return new_pos
            attempts += 1
        raise ValueError("Unable to find a new position for the given orientation")

    m, n = grid.shape
    new_grid = np.copy(grid)
    new_cells_coords = set()
    occupied = set(cells_coords)  # Initialize with original occupied positions

    for cell in cells_coords:
        row, col = cell
        color = grid[row, col]
        if color in orientation_requirements:
            orientation = orientation_requirements[color]
            try:
                new_row, new_col = get_new_position(m, n, orientation, occupied)
                new_grid[new_row, new_col] = color
                new_cells_coords.add((new_row, new_col))
                occupied.add((new_row, new_col))
            except ValueError:
                new_grid[row, col] = color
                new_cells_coords.add((row, col))
        else:
            new_grid[row, col] = color
            new_cells_coords.add((row, col))

    # Remove original positions
    for cell in cells_coords:
        row, col = cell
        new_grid[row, col] = 0

    return new_grid, new_cells_coords

# Example usage
initialized_grid = np.array([
    [ 0,  0,  0,  0,  1, -1, -1],
    [ 0,  0,  0,  0,  0,  5,  3],
    [ 0,  0,  2,  0,  4,  0, -1],
    [-1, -1,  0,  0, -1, -1, -1]
])

# Identify initial cells' coordinates
cells_coords = {(0, 4), (1, 5), (1, 6), (2, 2), (2, 4)}

# Apply the relocation
coloring_grid, new_cells_coords = relocate_by_orientation(initialized_grid, cells_coords)

print("Original grid:")
print(initialized_grid)
print("\nNew grid after relocation:")
print(coloring_grid)
print("\nNew cells coordinates:")
print(new_cells_coords)


Original grid:
[[ 0  0  0  0  1 -1 -1]
 [ 0  0  0  0  0  5  3]
 [ 0  0  2  0  4  0 -1]
 [-1 -1  0  0 -1 -1 -1]]

New grid after relocation:
[[ 5  0  0  1  0 -1 -1]
 [ 0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0 -1]
 [-1 -1  2  0 -1 -1 -1]]

New cells coordinates:
{(0, 0), (0, 3), (2, 0), (1, 6), (3, 2)}


In [259]:
import numpy as np
import random

# Function to randomly place k colors within the floorplan
def place_k_colors_on_grid(grid_arr, k):
    colors_placed = 0
    cells_coords = set()
    coloring_grid = grid_arr.copy()
    m, n = grid_arr.shape
    while colors_placed < k:
        row, col = random.randint(0, m - 1), random.randint(0, n - 1)
        if coloring_grid[row, col] == 0:  # Ensure the cell is within the floorplan and uncolored
            coloring_grid[row, col] = colors_placed + 1
            cells_coords.add((row, col))
            colors_placed += 1
    return coloring_grid, cells_coords

# Function to relocate cells based on orientation requirements
def relocate_by_orientation2(grid, cells_coords):

    def is_position_correct(cell_positions, color, position):
        row, col = position
        orientation = orientation_requirements[color]
        
        for other_color, other_position in cell_positions.items():
            if other_color == color:
                continue
            other_row, other_col = other_position
            
            if orientation == 'north' and row > other_row:
                return False
            if orientation == 'south' and row < other_row:
                return False
            if orientation == 'east' and col < other_col:
                return False
            if orientation == 'west' and col > other_col:
                return False
        return True
    
def relocate_by_orientation(grid, cells_coords):


    def is_position_correct(cell_positions, color, position):
        print(f'cell_position={cell_positions}')
        row, col = position
        orientation = orientation_requirements[color]
        print(f'color={color}, orientation = {orientation}')
        
        for other_color, other_position in cell_positions.items():
            if other_color == color:
                print(f'other_color==color: {color}')
                continue
            other_row, other_col = other_position
            
            if orientation == 'north' and row > other_row:
                return False
            if orientation == 'south' and row < other_row:
                return False
            if orientation == 'east' and col < other_col:
                return False
            if orientation == 'west' and col > other_col:
                return False
        return True

    def swap_positions(grid, pos1, pos2):
        grid[pos1[0], pos1[1]], grid[pos2[0], pos2[1]] = grid[pos2[0], pos2[1]], grid[pos1[0], pos1[1]]


    m, n = grid.shape
    new_grid = np.copy(grid)
    cell_positions = {grid[row, col]: (row, col) for row, col in cells_coords}

    any_changes = True
    while any_changes:
        any_changes = False
        for color, position in list(cell_positions.items()):
            if color in orientation_requirements and not is_position_correct(cell_positions, color, position):
                for other_color, other_position in list(cell_positions.items()):
                    if other_color != color and is_position_correct(cell_positions, other_color, position):
                        swap_positions(new_grid, position, other_position)
                        cell_positions[color], cell_positions[other_color] = other_position, position
                        any_changes = True
                        
    new_cells_coords = set(cell_positions.values())
    return new_grid, new_cells_coords                        
# Example usage
initialized_grid = np.array([
    [ 0,  0,  0,  0,  1, -1, -1],
    [ 0,  0,  0,  0,  0,  5,  3],
    [ 0,  0,  2,  0,  4,  0, -1],
    [-1, -1,  0,  0, -1, -1, -1]
])
orientation_requirements = {
    1: 'north',
    2: 'south',
    3: 'east',
    4: 'west',
    5: 'north'
}

# Identify initial cells' coordinates
cells_coords = {(0, 4), (1, 5), (1, 6), (2, 2), (2, 4)}

# Apply the relocation
coloring_grid, new_cells_coords = relocate_by_orientation(initialized_grid, cells_coords)

print("Original grid:")
print(initialized_grid)
print("\nNew grid after relocation:")
print(coloring_grid)
print("\nNew cells coordinates:")
print(new_cells_coords)


cell_position={4: (2, 4), 1: (0, 4), 2: (2, 2), 5: (1, 5), 3: (1, 6)}
color=4, orientation = west
other_color==color: 4
cell_position={4: (2, 4), 1: (0, 4), 2: (2, 2), 5: (1, 5), 3: (1, 6)}
color=1, orientation = north
other_color==color: 1
cell_position={4: (2, 4), 1: (0, 4), 2: (2, 2), 5: (1, 5), 3: (1, 6)}
color=2, orientation = south
other_color==color: 2
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=5, orientation = north
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=3, orientation = east
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=1, orientation = north
other_color==color: 1
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=2, orientation = south
other_color==color: 2
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=5, orientation = north
cell_position={4: (2, 2), 1: (0, 4), 2: (2, 4), 5: (1, 5), 3: (1, 6)}
color=4, orientation = west
ot